## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [0]:
import tensorflow as tf
tf.enable_eager_execution()

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [67]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data1 = pd.read_csv("/gdrive/My Drive/Colab Notebooks/DeepLearning/LAB6/prices.csv")

### Check all columns in the dataset

In [70]:
data1.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

### Drop columns `date` and  `symbol`

In [0]:
data1.drop(columns=['date','symbol'], inplace=True)


In [72]:
data1.columns

Index(['open', 'close', 'low', 'high', 'volume'], dtype='object')

In [73]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


In [74]:
data1.shape

(851264, 5)

In [75]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 5 columns):
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5)
memory usage: 32.5 MB


In [76]:
data1.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
open,851264.0,7.083699e+01,8.369588e+01,0.85,3.384000e+01,5.277000e+01,7.988000e+01,1.584440e+03
close,851264.0,7.085711e+01,8.368969e+01,0.86,3.385000e+01,5.280000e+01,7.989000e+01,1.578130e+03
low,851264.0,7.011841e+01,8.287729e+01,0.83,3.348000e+01,5.223000e+01,7.911000e+01,1.549940e+03
high,851264.0,7.154348e+01,8.446550e+01,0.88,3.419000e+01,5.331000e+01,8.061000e+01,1.600930e+03
volume,851264.0,5.415113e+06,1.249468e+07,0.00,1.221500e+06,2.476250e+06,5.222500e+06,8.596434e+08


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data=data1.head(1000)

In [0]:
import numpy as np

In [79]:
data['volume'] = [np.divide(x,1000000) for x in data['volume']]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [80]:
data['volume'][0]

2.1636

In [81]:
data1['volume'][0]

2.1636

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X= np.array(data.drop(columns='volume'))
y= np.array(data['volume'])

In [0]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=2324)

#### Convert Training and Test Data to numpy float32 arrays


In [85]:
X_train.shape

(800, 4)

In [86]:
X_test.shape

(200, 4)

In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')
y_train = y_train.astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
scale = Normalizer()

In [0]:
X_train_z = scale.fit_transform(X_train)

In [0]:
X_test_z= scale.fit_transform(X_test)

In [92]:
X_train_z.shape

(800, 4)

In [0]:
X_train_z = X_train_z.astype('float32')
X_test_z = X_test_z.astype('float32')
y_test = y_test.astype('float32')
y_train = y_train.astype('float32')

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias with Zero
w = tf.zeros(shape=(4,1))
b = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w, b):
    
    xw_matmul = tf.matmul(x, w)
    y = tf.add(xw_matmul, b)
    
    return y

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(x, y_actual, w, b, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w,b])
        
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    #Update Weights and Bias
    w = w - learning_rate*dw
    b = b - learning_rate*db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [98]:
   for i in range(100):
    
    w, b = train(X_train_z, y_train, w, b)
    print('Current Loss on iteration', i, loss(y_train, prediction(X_train_z, w, b)).numpy())

Current Loss on iteration 0 195.37025
Current Loss on iteration 1 193.4514
Current Loss on iteration 2 191.68297
Current Loss on iteration 3 190.0532
Current Loss on iteration 4 188.5512
Current Loss on iteration 5 187.16695
Current Loss on iteration 6 185.89122
Current Loss on iteration 7 184.7155
Current Loss on iteration 8 183.63194
Current Loss on iteration 9 182.63338
Current Loss on iteration 10 181.71306
Current Loss on iteration 11 180.8649
Current Loss on iteration 12 180.08325
Current Loss on iteration 13 179.36287
Current Loss on iteration 14 178.69897
Current Loss on iteration 15 178.08711
Current Loss on iteration 16 177.52322
Current Loss on iteration 17 177.00356
Current Loss on iteration 18 176.52463
Current Loss on iteration 19 176.08322
Current Loss on iteration 20 175.67644
Current Loss on iteration 21 175.30154
Current Loss on iteration 22 174.95604
Current Loss on iteration 23 174.63762
Current Loss on iteration 24 174.34418
Current Loss on iteration 25 174.07373
C

### Get the shapes and values of W and b

In [99]:
#Check Weights and Bias
print('Weights:\n', w.numpy())
print('Bias:\n',b.numpy())

Weights:
 [[1.2649972]
 [1.2692019]
 [1.251856 ]
 [1.2802608]]
Bias:
 [2.533298]


### Model Prediction on 1st Examples in Test Dataset

In [100]:
X_test_z[0]

array([0.50227785, 0.49966323, 0.49195227, 0.50600046], dtype=float32)

In [101]:
X_test_z.shape

(200, 4)

In [0]:
prediction = prediction(X_test_z[0:1], w,b)

In [103]:
prediction

<tf.Tensor: id=14193, shape=(1, 1), dtype=float32, numpy=array([[5.066518]], dtype=float32)>

In [104]:
y_test[0]

0.4599

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
data2 = pd.read_csv("/gdrive/My Drive/Colab Notebooks/DeepLearning/LAB6/Iris.csv")

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [106]:
data2.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
from sklearn import preprocessing 
  
# label_encoder object knows how to understand word labels. 
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
data2['Species']= label_encoder.fit_transform(data2['Species']) 

In [108]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
Id               150 non-null int64
SepalLengthCm    150 non-null float64
SepalWidthCm     150 non-null float64
PetalLengthCm    150 non-null float64
PetalWidthCm     150 non-null float64
Species          150 non-null int64
dtypes: float64(4), int64(2)
memory usage: 7.2 KB


### Splitting the data into feature set and target set

In [0]:
X1= np.array(data2.drop(columns=['Species','Id']))

In [0]:
y1 = np.array(data2['Species'])

In [0]:
X1_train,X1_test,y1_train,y1_test = train_test_split(X1,y1,test_size=0.2,random_state=2324)

In [0]:
X1_train_z = scale.fit_transform(X1_train)

In [0]:
X1_test_z = scale.fit_transform(X1_test)

In [0]:
y1_train = tf.keras.utils.to_categorical(y1_train, num_classes=3)
y1_test = tf.keras.utils.to_categorical(y1_test, num_classes=3)


In [115]:

print(y1_train.shape)
print('First 2 examples now are: ', y1_train[0:2])

(120, 3)
First 2 examples now are:  [[0. 0. 1.]
 [0. 1. 0.]]


In [0]:
X1_train_z = X1_train_z.astype('float32')
X1_test_z = X1_test_z.astype('float32')
y1_test = y1_test.astype('float32')
y1_train = y1_train.astype('float32')

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [117]:
X1_test_z.shape

(30, 4)

In [0]:
#Initialize Sequential model
model = tf.keras.models.Sequential()

In [0]:

#Normalize the data
model.add(tf.keras.layers.BatchNormalization())

In [0]:
#Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, input_shape=(4,),activation='softmax'))



In [0]:

#Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', 
              metrics=['accuracy'])

### Model Training 

In [123]:

model.fit(X1_train_z, y1_train, 
          validation_data=(X1_test_z, y1_test), 
          epochs=100,
          batch_size=X1_train_z.shape[0])

Train on 120 samples, validate on 30 samples
Epoch 1/100
120/120 [==============================] - 1s 6ms/sample - loss: 1.1182 - acc: 0.3417 - val_loss: 1.2295 - val_acc: 0.3000
Epoch 2/100
120/120 [==============================] - 0s 131us/sample - loss: 1.1164 - acc: 0.3417 - val_loss: 1.2282 - val_acc: 0.3000
Epoch 3/100
120/120 [==============================] - 0s 111us/sample - loss: 1.1146 - acc: 0.3417 - val_loss: 1.2269 - val_acc: 0.3000
Epoch 4/100
120/120 [==============================] - 0s 145us/sample - loss: 1.1128 - acc: 0.3417 - val_loss: 1.2256 - val_acc: 0.3000
Epoch 5/100
120/120 [==============================] - 0s 111us/sample - loss: 1.1110 - acc: 0.3417 - val_loss: 1.2243 - val_acc: 0.3000
Epoch 6/100
120/120 [==============================] - 0s 118us/sample - loss: 1.1092 - acc: 0.3417 - val_loss: 1.2231 - val_acc: 0.3000
Epoch 7/100
120/120 [==============================] - 0s 108us/sample - loss: 1.1074 - acc: 0.3417 - val_loss: 1.2218 - val_acc: 0.300

### Model Prediction

In [124]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo multiple                  16        
_________________________________________________________________
dense (Dense)                multiple                  15        
_________________________________________________________________
dense_1 (Dense)              multiple                  12        
Total params: 43
Trainable params: 35
Non-trainable params: 8
_________________________________________________________________


### Save the Model

In [0]:
model.save('mnist_lc.h5')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?